In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# visualise 5 random rows from the iris dataset
data = pd.read_csv("iris.csv")
data['species'] = data['species'].astype('category').cat.codes
data.iloc[np.random.permutation(data.shape[0])[:5]]

,sepal_length,sepal_width,petal_length,petal_width,species
17,5.1,3.5,1.4,0.3,0
131,7.9,3.8,6.4,2.0,2
18,5.7,3.8,1.7,0.3,0
5,5.4,3.9,1.7,0.4,0
93,5.0,2.3,3.3,1.0,1


In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier

In [4]:
# possible hidden layer sizes to consider
# single layer
hidden_layer_sizes = [(i, ) for i in range(5, 15)]

# two layers
hidden_layer_sizes.extend([(i, j) for i in range(5, 15, 3) for j in range(5, 15, 3)])

hidden_layer_sizes

[(5,),
 (6,),
 (7,),
 (8,),
 (9,),
 (10,),
 (11,),
 (12,),
 (13,),
 (14,),
 (5, 5),
 (5, 8),
 (5, 11),
 (5, 14),
 (8, 5),
 (8, 8),
 (8, 11),
 (8, 14),
 (11, 5),
 (11, 8),
 (11, 11),
 (11, 14),
 (14, 5),
 (14, 8),
 (14, 11),
 (14, 14)]

In [5]:
MAX_ITER = 5000

In [6]:
# perform 5-fold cross validation to find the best hidden layer size from the options
res = GridSearchCV(MLPClassifier(max_iter = MAX_ITER), param_grid={'hidden_layer_sizes': hidden_layer_sizes}, cv=5).fit(data.iloc[:, 0:4], data.iloc[:, 4])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,0:4], data.iloc[:,4], test_size=0.25, random_state=0)
pd.concat([X_test, y_test], axis = 1).to_csv("iris_test.csv", index = False)

In [8]:
res.best_params_, res.best_score_, res.score(X_test, y_test), res.classes_

({'hidden_layer_sizes': (7,)},
 0.9866666666666667,
 0.9736842105263158,
 array([0, 1, 2], dtype=int8))

In [9]:
# get best 5 models
best = res.cv_results_['rank_test_score'].argsort()
indices = np.append(best[:5], best[-1])
params = [d['hidden_layer_sizes'] for d in np.array(res.cv_results_['params'])[indices]]
params.extend([(5, 11, 8), tuple([np.random.randint(5, 30) for i in range(12)])])
params

[(7,),
 (11, 8),
 (10,),
 (5,),
 (11, 14),
 (8, 5),
 (5, 11, 8),
 (9, 14, 7, 26, 27, 12, 14, 6, 14, 17, 16, 27)]

In [38]:
import pickle
from struct import pack

for p in params:
	print(f"training model with hidden layer sizes {p}")
	model = MLPClassifier(hidden_layer_sizes = p, max_iter = MAX_ITER).fit(X_train, y_train)
	print(f"accuracy: {model.score(X_test, y_test)}")

	fname = f'trained_iris_model_{"_".join([str(s) for s in p])}'
	with open(f"models/{fname}.pkl", "wb") as f:
		pickle.dump(model, f)
	
	layers = len(model.coefs_)
	# number of cols of weights matrix, this is pre transpose
	input_neurons = model.coefs_[0].shape[0]

	with open(f"weights/{fname}.mdl", 'wb') as f:
		f.write(pack('I', layers))
		f.write(pack('I', input_neurons))
		for i in range(layers):
			weights = model.coefs_[i].transpose()
			f.write(pack('I', weights.shape[0]))
			weights = list(weights.flatten())
			f.write(pack('d' * len(weights), *weights))
			biases = list(model.intercepts_[i])
			f.write(pack('d' * len(biases), *biases))
	
	print()

training model with hidden layer sizes (7,)
accuracy: 0.9736842105263158

training model with hidden layer sizes (11, 8)
accuracy: 0.9736842105263158

training model with hidden layer sizes (10,)
accuracy: 0.9736842105263158

training model with hidden layer sizes (5,)
accuracy: 0.9736842105263158

training model with hidden layer sizes (11, 14)
accuracy: 0.9736842105263158

training model with hidden layer sizes (8, 5)
accuracy: 0.9736842105263158

training model with hidden layer sizes (5, 11, 8)
accuracy: 0.9736842105263158

training model with hidden layer sizes (9, 14, 7, 26, 27, 12, 14, 6, 14, 17, 16, 27)
accuracy: 0.9736842105263158

